The structure of the files is as follows:


number of customers
number of available depots

coordinates for the depots (x and y)

coordinates for the customers

vehicle capacity

depot capacities (for Tuzun instances, each one is equal to the total demand as there is no capacity on the depots)

customers demands

opening costs for the depots

opening cost of a route (cost of a vehicle)

0 or 1 (0 means that the costs are integer - 1 that costs are real)


To calculate the matrix distance (or the cost to link any 2 points A and B in the graph), we use the mathematical formula:

sqrt( (xA-xB)² + (yA-yB)² )

The results are stored in a float variable (in C language) if the costs are real (code 1)
The result is multiplied by 100 and truncked to be stored in an integer variable if the costs are interger (code 0).

Example of a file:

20
5

6	7
19	44
37	23
35	6
5	8

20	35
8	31
29	43
18	39
19	47
31	24
38	50
33	21
2	27
1	12
26	20
20	33
15	46
20	26
17	19
15	12
5	30
13	40
38	5
9	40

70

140
140
140
140
140

17
18
13
19
12
18
13
13
17
20
16
18
15
11
18
16
15
15
15
16

10841
11961
6091
7570
7497

1000

0



In [1]:
from docplex.mp.model import Model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

Parser

In [2]:
def parse_file(file_path):
    with open(file_path, 'r') as file:
        lines = [line.strip() for line in file if line.strip()]

    num_customers, num_depots = int(lines[0]), int(lines[1])
    depots = [tuple(map(float, lines[i].split())) for i in range(2, 2 + num_depots)]
    customers = [tuple(map(float, lines[i].split())) for i in range(2 + num_depots, 2 + num_depots + num_customers)]
    vehicle_capacity = float(lines[2 + num_depots + num_customers])
    depot_capacities = list(map(float, lines[3 + num_depots + num_customers:3 + 2 * num_depots + num_customers]))
    customer_demands = list(
        map(float, lines[3 + 2 * num_depots + num_customers:3 + 2 * num_depots + 2 * num_customers]))
    depot_opening_costs = list(
        map(float, lines[3 + 2 * num_depots + 2 * num_customers:3 + 3 * num_depots + 2 * num_customers]))
    route_opening_cost = float(lines[3 + 3 * num_depots + 2 * num_customers])

    all_points = depots + customers
    distance_matrix = [
        [math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2) for x2, y2 in all_points] for x1, y1 in all_points
    ]

    return {
        "num_customers": num_customers,
        "num_depots": num_depots,
        "depots": depots,
        "customers": customers,
        "vehicle_capacity": vehicle_capacity,
        "depot_capacities": depot_capacities,
        "customer_demands": customer_demands,
        "depot_opening_costs": depot_opening_costs,
        "route_opening_cost": route_opening_cost,
        "distance_matrix": distance_matrix,
    }

In [3]:
def modelo_scf(parsed_data):

    n = parsed_data["num_customers"]
    m = parsed_data["num_depots"]
    depots = parsed_data["depots"]
    customers = parsed_data["customers"]
    vehicle_capacity = parsed_data["vehicle_capacity"]
    depot_capacities = parsed_data["depot_capacities"]
    customers_demands = parsed_data["customer_demands"]
    opening_costs_depots = parsed_data["depot_opening_costs"]
    costs = parsed_data["distance_matrix"]


    mdl = Model(name="SCF")

    # Variables
    nodos = list(range(n + m))
    x = mdl.binary_var_matrix(nodos, nodos, name="x")  # Ruta entre nodos
    f = mdl.continuous_var_matrix(nodos, nodos, name="f")  # Flujo
    y = mdl.binary_var_list(m, name="y")  # Uso de depósitos

    # Función objetivo
    mdl.minimize(
        mdl.sum(costs[i][j] * x[i, j] for i in nodos for j in nodos if i != j) +
        mdl.sum(opening_costs_depots[d] * y[d] for d in range(m))
    )

    # Restricciones
    # 1. Cada cliente es atendido exactamente una vez
    for i in range(m, m + n):
        mdl.add_constraint(mdl.sum(x[i, j] for j in nodos if j != i) == 1)
        mdl.add_constraint(mdl.sum(x[j, i] for j in nodos if j != i) == 1)

    # 2. Cada ruta debe comenzar y terminar en un depósito
    for d in range(m):
        mdl.add_constraint(mdl.sum(x[d, j] for j in range(m, m + n)) <= y[d])
        mdl.add_constraint(mdl.sum(x[j, d] for j in range(m, m + n)) <= y[d])

    # 3. Restricciones de flujo
    for i in range(m, m + n):
        mdl.add_constraint(
            mdl.sum(f[i, j] for j in nodos if j != i) -
            mdl.sum(f[j, i] for j in nodos if j != i) == customers_demands[i - m]
        )
    for i, j in [(i, j) for i in nodos for j in nodos if i != j]:
        mdl.add_constraint(f[i, j] <= vehicle_capacity * x[i, j])

    # 4. Capacidades de los depósitos
    for d in range(m):
        mdl.add_constraint(
            mdl.sum(f[d, j] for j in range(m, m + n)) <= depot_capacities[d]
        )

    return mdl


In [4]:
file_path = 'Instances/Benchmark_1/coord20-5-1.dat'

parsed_data = parse_file(file_path)

In [5]:
mdl = modelo_scf(parsed_data)
mdl.print_information()
mdl.solve(log_output=True)

Model: SCF
 - number of variables: 1255
   - binary=630, integer=0, continuous=625
 - number of constraints: 675
   - linear=675
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 20 rows and 90 columns.
Reduced MIP has 655 rows, 1165 columns, and 3390 nonzeros.
Reduced MIP has 585 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (1.89 ticks)
Probing time = 0.02 sec. (2.93 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 655 rows, 1165 columns, and 3390 nonzeros.
Reduced MIP has 585 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (2.33 ticks)
Probing time = 0.03 sec. (2.86 ticks)
Clique table members: 50.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.


docplex.mp.solution.SolveSolution(obj=44212.6,values={x_0_20:1,x_1_22:1,..

In [7]:
def plot_solution(mdl, x):
    n = len(x)
    m = int(math.sqrt(n))
    plt.figure(figsize=(10, 10))
    for i in range(m):
        for j in range(m):
            if x[i * m + j] == 1:
                plt.scatter(depots[i][0], depots[i][1], c='r', s=100)
                plt.text(depots[i][0], depots[i][1], f'D{i}', fontsize=12)
            if x[i * m + j] == 2:
                plt.scatter(customers[j - m][0], customers[j - m][1], c='b', s=50)
                plt.text(customers[j - m][0], customers[j - m][1], f'C{j - m}', fontsize=12)
    plt.show()

In [8]:
x = mdl.solution.get_all_values()
plot_solution(mdl, x)

AttributeError: 'SolveSolution' object has no attribute 'get_all_values'